# **크롤링(Crawling)**

## **1. 크롤링 개요**

- 크롤링이란?
  - 웹 페이지를 그대로 다운로드하여 그 속에서 데이터를 추출하는 행위
  - 웹에 존재하는 사이트를 방문하여 그 내용의 복사본을 저장하고, 수집된 정보들은 재가공하여 사용자가 원하는 검색 결과를 제공함
  - 스크레이핑(Scraping)이라고도 부름
- 크롤링의 방법
  1. 웹페이지에서 정보를 다운로드 → (파이썬에서는) requests 라이브러리 사용
  2. HTML 소스코드를 파싱(분석)하여 원하는 정보 획득 → (파이썬에서는) BeautifulSoup 라이브러리 사용
- 원활한 크롤링을 위해서는 HTML 문서의 구조를 잘 알아둘 필요가 있음

## **2. BeautifulSoup 사용하기**

In [1]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=8262fbbeeb0825f21f99797aa94764b8f7c5797150f203c577ee079724887a71
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [2]:
from bs4 import BeautifulSoup

In [3]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [4]:
# Beautifulsoup 객체를 생성한다.
soup = BeautifulSoup(html_doc,'lxml')
# 객체.태그 이름
# .태그 이름으로 하위 태그로의 접근이 가능하다.
print("soup.body.p의 결과 : ", soup.body.p)

soup.body.p의 결과 :  <p class="title"><b>The Dormouse's story</b></p>


In [5]:
# 객체.태그['속성 이름']
# 객체의 태그 속성은 파이썬 딕셔너리처럼 태그['속성 이름']으로 접근이 가능하다.
print("soup.a['href']의 결과 : ", soup.a['href'])

soup.a['href']의 결과 :  http://example.com/elsie


In [6]:
# 객체.name
## name 변수
print("soup.title.name의 결과 : ", soup.title.name)

soup.title.name의 결과 :  title


In [7]:
# 객체.string
## string 변수 (참고) NavigableString: 문자열은 태그 안의 텍스트에 상응한다. BeautifulSoup은 이런 텍스트를 포함하는 NavigableString 클래스를 사용한다.
print("soup.title.string의 결과 : ", soup.title.string)

soup.title.string의 결과 :  The Dormouse's story


In [8]:
# 객체.contents
## 태그의 자식들을 리스트로 반환
print("soup.contents의 결과 : ", soup.contents)

soup.contents의 결과 :  [<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
</body></html>]


In [9]:
# find() : 태그 하나만 가져옴
'''
find(name, attrs, recursive, string, **kwargs)
[옵션]
name – 태그 이름
attrs – 속성(딕셔너리로)
recursive – 모든 자식 or 자식
string – 태그 안에 텍스트
keyword – 속성(키워드로)
※ (주의) class는 파이썬 예약어이므로, class_를 사용한다.
'''
print("soup.find() 의 결과 : ", soup.find('a', attrs={'class' : 'sister'}))

soup.find() 의 결과 :  <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>


In [10]:
# find_all() : 해당 태그가 여러 개 있을 경우 한꺼번에 모두 가져온다. 그 객체들의 리스트로 반환한다. 
'''
find_all(name, attrs, recursive, string, limit, **kwargs)
[옵션]
Limit –몇 개까지 찾을 것인가? find_all()로 검색했을 때, 수천, 수만 개가 된다면 시간이 오래 걸릴 것이다. 
이때 몇 개까지만 찾을 수 있도록 제한을 둘 수 있는 인자다.
'''
print("soup.find_all()의 결과 : ", soup.find_all('a'))

soup.find_all()의 결과 :  [<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]


## **3. 네이버 뉴스 크롤링하기**

In [15]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime

In [16]:
# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27'}
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')

    return soup

In [17]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info(sec, sid):
    # 임시 이미지
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"

    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=" + sid
    print("section url : ", sec_url)

    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)

    # 해당 분야 상위 뉴스 3개 가져오기
    news_list3 = []
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
        news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") ,
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" : li.img.attrs.get('src') if li.img else default_img
        }
        news_list3.append(news_info)

    return news_list3

In [18]:
# 뉴스 본문 가져오기
def get_news_contents(url):
    soup = get_soup_obj(url)
    # body = soup.find('div', class_="_article_body_contents")
    body = soup.find('div', class_="go_trans _article_content")  # 현재는 Naver의 기사 본몬을 표시하는 class 명이 변경되었음
    
    news_contents = ''
    for content in body:
        if type(content) is bs4.element.NavigableString and len(content) > 50:
            # content.strip() : whitepace 제거 (참고 : https://www.tutorialspoint.com/python3/string_strip.htm)
            # 뉴스 요약을 위하여 '.' 마침표 뒤에 한칸을 띄워 문장을 구분하도록 함
            news_contents += content.strip() + ' '

    return news_contents

In [19]:
# '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    # 뉴스 결과를 담아낼 dictionary
    news_dic = dict()

    # sections : '정치', '경제', '사회'
    sections = ["pol", "eco","soc"]
    # section_ids : URL에 사용될 뉴스 각 부문 ID
    section_ids = ["100", "101","102"]

    for sec, sid in zip(sections, section_ids):
        # 뉴스의 기본 정보 가져오기
        news_info = get_top3_news_info(sec, sid)
        #print(news_info)
        for news in news_info:
            # 뉴스 본문 가져오기
            news_url = news['news_url']
            news_contents = get_news_contents(news_url)

            # 뉴스 정보를 저장하는 dictionary를 구성
            news['news_contents'] = news_contents

        news_dic[sec] = news_info

    return news_dic

In [20]:
# 함수 호출 - '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
news_dic = get_naver_news_top3()
# 경제의 첫번째 결과 확인하기
news_dic['eco'][0]

section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100
section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101
section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=102


{'title': ' [fn마감시황]코스피, 동학개미 5510억 순매도..2520선 상승세',
 'date': '1분전',
 'news_url': 'https://n.news.naver.com/mnews/article/014/0005006086?sid=101',
 'image_url': 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#',
 'news_contents': '[파이낸셜뉴스] 코스피가 동학개미로 불리는 개인투자자의 5510억원을 넘는 순매도세에도 2520선에서 상승세다. 하지만 외국인, 기관, 금융투자, 연기금 등은 각각 3418억원, 2282억원, 1917억원, 202억원을 순매수하며 지수를 끌어올렸다. 규모별로 대형주(0.84%), 중형주(1.19%), 소형주(1.57%) 등 모두 상승세다. '}